In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name



/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [2]:
import os
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1'

In [5]:
def load_csv_dataset(file_path):
    return pd.read_csv(file_path)


cyrillic_guys = load_csv_dataset('cyrillic_dudes.csv')
lookup_guys = load_csv_dataset('lookup_required_dudes.csv')

online_lookup_1_found = load_csv_dataset('Online lookup required - 1 name found.csv')
online_lookup_0_found = load_csv_dataset('Online lookup required - 0 names found.csv')
online_lookup_multi_found = load_csv_dataset('Online lookup required - multiple names found.csv')

# Cyrillic

# Lookup Required

# Online Lookup Required

In [ ]:
#online lookup - 1 guy found 

#if the name is initial + name make sure the name starts with initial and ends with name. be mindful of the accents here if names are same w accents removed that's ok

#if its just name (no initials) do nothing 

#is the name that it found the closest match in the dataset_nationality?
#is there another name in the dataset thats basically a better quality of match?? or the same? 
    #if there is another potential name do you do the multi name match date lookup? 

        #if one guy is remaining YOU HAVE A MATCH

        #if multiple remaining, then see the below example when there's an array of names

        #if 0 remaining then what?

    #if not any other potential names, evaluate the quality of this match. 

        #if the quality is good then YOU HAVE A MATCH

        #if the quality is super super low then what?  

    #provided you have a name 
    #look up that name + that season 
    #if there are no rows, you have to look them up online 

In [ ]:
#online lookup required - multiple names found 


#if multiple names were found

    #if the Jersey has an initial 
        #keep all matches that start with the initial 

        #except if the initial is a lowercase d. then, make sure there is a word starting with d followed by the Name 
        #so if its d Silva make sure theres a word startign with d then Silva soemwhere in the name. 

    #sadiqov case - what if the names are the same???
        #in this case you should just look up each name and use like whichever has the record from the saeason of the game 

    #do the match date lookup thing with each of the matches to check who played in that game. 
        #if you still have more than 1 name

            #look at the success dataset to find the record of each remaining guy
                #if its ivanovic and you find both B and D Ivanovic played according to transfermarkt
                
                #check if there's a record of those guys in that match / date (in our dataset)
                    
                    #if all but one guy is in the success dataset, the guy who is not there is the name you need to use.

                    #if none remain after checking the dataset just remove this row. but print that out. that would be weird

                    #if multiple remain after checking the success dataset(there were 3 names remaining, only found 1 record)
                        #print this out this is a special case.

        #if you are just left with one name using the match date lookup 

            #that's a success - find that guy in the database